# Custom Chat Message Store for Multi-Turn Conversations

This notebook demonstrates how to implement and use a **custom chat message store** for managing conversation threads with AI agents using the Microsoft Agent Framework.

## Overview

The Agent Framework provides flexibility in how conversation history is stored and managed. While the framework includes default in-memory storage, you can implement custom storage solutions to:

- **Persist conversations** to databases (SQL, NoSQL, vector stores)
- **Integrate with external systems** for conversation management
- **Apply custom logic** for message retention and retrieval
- **Scale across distributed systems** with centralized storage

## Key Concepts

### ChatMessageStoreProtocol

Custom message stores must implement the `ChatMessageStoreProtocol`, which defines:
- `add_messages()`: Store new messages in the conversation
- `list_messages()`: Retrieve conversation history
- `serialize_state()`: Convert store state to a serializable format
- `deserialize_state()`: Restore store state from serialized data

### Thread Serialization

For in-memory threads with custom stores:
- The serialized thread contains the **complete message history**
- Allows conversation context to be saved and resumed across sessions
- Useful for stateless applications or conversation backups

## 📖 Documentation

For more details, see the official documentation:
- [Multi-Turn Conversations](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python)
- [Custom Message Stores](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python#custom-message-stores)

---

## Prerequisites

Before running this notebook, ensure you have:
1. **Environment variables configured** in `agent-framework/.env`
2. **OpenAI API key** set (or other chat client credentials)
3. **Required packages installed** (see imports below)

## Setup and Imports

In [13]:
# Import required libraries
import asyncio
from collections.abc import Collection
from typing import Any

from agent_framework import ChatMessage, ChatMessageStoreProtocol
from pydantic import BaseModel

In [14]:
# Load environment variables from agent-framework/.env
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from the agent-framework directory
env_path = Path(__file__).parent.parent / ".env" if "__file__" in globals() else Path("../.env")
load_dotenv(env_path)

# Verify key environment variables are loaded
print("Environment variables loaded:")
print(f"  OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")
print(f"  OPENAI_CHAT_MODEL_ID: {os.getenv('OPENAI_CHAT_MODEL_ID', 'Not set - using gpt-4o as default')}")
print()

Environment variables loaded:
  OPENAI_API_KEY: ✓ Set
  OPENAI_CHAT_MODEL_ID: gpt-4o



## Implementing a Custom Chat Message Store

We'll create a custom message store that demonstrates the pattern. In production, you would replace the in-memory list with actual database calls (e.g., PostgreSQL, MongoDB, Cosmos DB, or a vector database).

### State Model

First, we define a Pydantic model to represent the serializable state of our store:

In [15]:
class CustomStoreState(BaseModel):
    """Implementation of custom chat message store state.
    
    This model defines what gets serialized when saving the conversation state.
    In a production system, this could include additional metadata like:
    - User ID
    - Session information
    - Timestamps
    - Custom tags or categories
    """
    model_config = {"arbitrary_types_allowed": True}
    
    messages: list[ChatMessage]

### Custom Store Implementation

Now we implement the `ChatMessageStoreProtocol` with our custom logic:

In [16]:
class CustomChatMessageStore(ChatMessageStoreProtocol):
    """Implementation of custom chat message store.
    
    In real applications, this can be an implementation of:
    - Relational database (PostgreSQL, MySQL, SQL Server)
    - NoSQL database (MongoDB, Cosmos DB, DynamoDB)
    - Vector store (Pinecone, Weaviate, Azure AI Search)
    - File storage (Azure Blob, S3)
    
    This example uses an in-memory list for demonstration purposes.
    """

    def __init__(self, messages: Collection[ChatMessage] | None = None) -> None:
        """Initialize the message store.
        
        Args:
            messages: Optional initial messages to populate the store
        """
        self._messages: list[ChatMessage] = []
        if messages:
            self._messages.extend(messages)

    async def add_messages(self, messages: Collection[ChatMessage]) -> None:
        """Add new messages to the store.
        
        In a production implementation, this would:
        - Insert messages into a database
        - Handle connection pooling
        - Implement retry logic
        - Add error handling
        
        Args:
            messages: Collection of messages to add
        """
        self._messages.extend(messages)

    async def list_messages(self) -> list[ChatMessage]:
        """Retrieve all messages from the store.
        
        In a production implementation, this would:
        - Query the database
        - Implement pagination for large conversations
        - Apply filtering or sorting
        
        Returns:
            List of all stored messages
        """
        return self._messages

    async def deserialize_state(self, serialized_store_state: Any, **kwargs: Any) -> None:
        """Restore the store state from serialized data.
        
        This method is called when resuming a conversation from saved state.
        
        Args:
            serialized_store_state: Previously serialized state data
            **kwargs: Additional arguments for deserialization
        """
        if serialized_store_state:
            state = CustomStoreState.model_validate(serialized_store_state, **kwargs)
            if state.messages:
                self._messages.extend(state.messages)

    async def serialize_state(self, **kwargs: Any) -> Any:
        """Serialize the current store state.
        
        This method is called when saving a conversation for later use.
        
        Args:
            **kwargs: Additional arguments for serialization
            
        Returns:
            Serialized state as a dictionary
        """
        state = CustomStoreState(messages=self._messages)
        return state.model_dump(**kwargs)

## Using the Custom Message Store

Now let's see how to use our custom message store with an AI agent. This example demonstrates:

1. **Creating an agent** with a custom message store factory
2. **Starting a conversation thread** using the custom store
3. **Serializing the thread** to save conversation state
4. **Deserializing the thread** to resume the conversation

### Agent Creation with Custom Store

We pass a factory function to the agent that creates instances of our custom store:

In [17]:
async def demonstrate_custom_store():
    """Demonstrates how to use 3rd party or custom chat message store for threads."""
    from openai import AsyncAzureOpenAI
    from agent_framework.openai import OpenAIChatClient
    
    print("=== Thread with Custom Chat Message Store ===")
    print()
    print("📌 Using Azure OpenAI for this example.")
    print()

    # Create Azure OpenAI async client
    azure_client = AsyncAzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY") or os.getenv("OPENAI_API_KEY"),
        api_version="2024-10-21",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )
    
    # Create an agent with Azure OpenAI client and custom message store factory
    agent = OpenAIChatClient(
        model_id=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o"),
        async_client=azure_client
    ).create_agent(
        name="Joker",
        instructions="You are good at telling jokes.",
        # Use custom chat message store.
        # If not provided, the default in-memory store will be used.
        chat_message_store_factory=CustomChatMessageStore,
    )

    print("✓ Agent created with custom message store")
    print()
    
    # Start a new thread for the agent conversation.
    # This will create a new instance of CustomChatMessageStore
    thread = agent.get_new_thread()
    
    print("✓ New thread created")
    print()

    # --- Phase 1: Initial Conversation ---
    print("--- Phase 1: Initial Conversation ---")
    print()
    
    query = "Tell me a joke about a pirate."
    print(f"User: {query}")
    response = await agent.run(query, thread=thread)
    print(f"Agent: {response}")
    print()

    # --- Phase 2: Suspend (Serialize) ---
    print("--- Phase 2: Suspending Conversation ---")
    print()
    
    # Serialize the thread state, so it can be stored for later use.
    # For custom stores, this includes the entire message history
    serialized_thread = await thread.serialize()

    print("📦 Serialized thread (first 200 chars):")
    print(f"   {str(serialized_thread)[:200]}...")
    print()
    print("💾 The thread can now be saved to:")
    print("   - Database (PostgreSQL, MongoDB, etc.)")
    print("   - File storage (JSON, Azure Blob, S3)")
    print("   - Cache (Redis, Memcached)")
    print("   - Any other storage mechanism")
    print()

    # Simulate loading the thread from storage and resuming the conversation
    print("--- Phase 3: Resuming Conversation ---")
    print()
    
    # Deserialize the thread state after loading from storage.
    # This creates a new thread with the same conversation history
    resumed_thread = await agent.deserialize_thread(serialized_thread)
    
    print("✓ Thread deserialized and conversation context restored")
    print()

    # Continue the conversation - the agent remembers the previous joke
    query = "Now tell the same joke in the voice of a pirate, and add some emojis to the joke."
    print(f"User: {query}")
    response = await agent.run(query, thread=resumed_thread)
    print(f"Agent: {response}")
    print()
    
    print("✅ Conversation resumed successfully with full context!")

## Run the Demo

Execute the demonstration to see the custom message store in action:

In [18]:
# Run the demonstration
await demonstrate_custom_store()

=== Thread with Custom Chat Message Store ===

📌 Using Azure OpenAI for this example.

✓ Agent created with custom message store

✓ New thread created

--- Phase 1: Initial Conversation ---

User: Tell me a joke about a pirate.
✓ Agent created with custom message store

✓ New thread created

--- Phase 1: Initial Conversation ---

User: Tell me a joke about a pirate.
Agent: Why did the pirate go to the gym?  

To improve his **plank**! 🏴‍☠️

--- Phase 2: Suspending Conversation ---

📦 Serialized thread (first 200 chars):
   {'service_thread_id': None, 'chat_message_store_state': None}...

💾 The thread can now be saved to:
   - Database (PostgreSQL, MongoDB, etc.)
   - File storage (JSON, Azure Blob, S3)
   - Cache (Redis, Memcached)
   - Any other storage mechanism

--- Phase 3: Resuming Conversation ---

✓ Thread deserialized and conversation context restored

User: Now tell the same joke in the voice of a pirate, and add some emojis to the joke.
Agent: Why did the pirate go to the gym

## Key Takeaways

1. **Custom Storage Flexibility**: Implement `ChatMessageStoreProtocol` to integrate with any storage backend

2. **Factory Pattern**: Use `chat_message_store_factory` to provide custom store instances for each thread

3. **Full Serialization**: For in-memory custom stores, serialized threads contain complete message history

4. **Production Considerations**:
   - Add database connection pooling
   - Implement error handling and retries
   - Consider message pagination for large conversations
   - Add indexes for efficient querying
   - Implement data retention policies

5. **Use Cases**:
   - Multi-tenant applications with isolated storage
   - Compliance requirements for conversation auditing
   - Integration with existing data infrastructure
   - Advanced analytics on conversation patterns

## Next Steps

- Explore **[Redis Chat Message Store](3-redis_chat_message_store_thread.ipynb)** for distributed scenarios
- Learn about **[Thread Suspend/Resume](4-suspend_resume_thread.ipynb)** patterns
- Review **[Multi-Turn Conversation Documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python)**